In [1]:
# Create the directory if it doesn't exist
!mkdir -p ../data

# Download files using curl
!curl -o ../data/green_tripdata_2024-01.parquet https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2024-01.parquet
!curl -o ../data/green_tripdata_2024-02.parquet https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2024-02.parquet

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1330k  100 1330k    0     0   300k      0  0:00:04  0:00:04 --:--:--  300k0:04 --:--:--  307k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1253k  100 1253k    0     0   566k      0  0:00:02  0:00:02 --:--:--  566k   192k      0  0:00:06  0:00:01  0:00:05  192k


In [1]:
import pickle
import pandas as pd
from sklearn.metrics import  root_mean_squared_error
from sklearn.feature_extraction import  DictVectorizer
from sklearn.linear_model import Lasso, Ridge, LinearRegression

In [2]:
def read_dataframe(filename):

    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df

In [3]:
df_train = read_dataframe('../data/green_tripdata_2024-01.parquet')
df_val = read_dataframe('../data/green_tripdata_2024-02.parquet')

In [4]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [5]:
categorical = ['PU_DO']  #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']
dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [6]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [14]:
import dagshub
import mlflow


dagshub.init(url="https://dagshub.com/daduke1/nyc-taxi-time-prediction", mlflow=True)

MLFLOW_TRACKING_URI = mlflow.get_tracking_uri()

print(MLFLOW_TRACKING_URI)

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment(experiment_name="nyc-taxi-experiment")

Initialized MLflow to track repo "daduke1/nyc-taxi-time-prediction"

Repository daduke1/nyc-taxi-time-prediction initialized!

https://dagshub.com/daduke1/nyc-taxi-time-prediction.mlflow.mlflow


<Experiment: artifact_location='mlflow-artifacts:/f141c1d9a6814959aec4b2f9fd9e18e0', creation_time=1726630127899, experiment_id='0', last_update_time=1726630127899, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [15]:
training_dataset = mlflow.data.from_numpy(X_train.data, targets=y_train, name="green_tripdata_2024-01")
validation_dataset = mlflow.data.from_numpy(X_val.data, targets=y_val, name="green_tripdata_2024-02")

In [16]:
from dagshub import get_repo_bucket_client
# Get a boto3.client object
s3 = get_repo_bucket_client("daduke1/nyc-taxi-time-prediction")

# Upload train file
s3.upload_file(
    Bucket="nyc-taxi-time-prediction",  # name of the repo
    Filename="../data/green_tripdata_2024-01.parquet",  # local path of file to upload
    Key="train_data.parquet",  # remote path where to upload the file
)

# Upload test file
s3.upload_file(
    Bucket="nyc-taxi-time-prediction",  # name of the repo
    Filename="../data/green_tripdata_2024-02.parquet",  # local path of file to upload
    Key="test_data.parquet",  # remote path where to upload the file
)

Client created. Use the name of the repo (nyc-taxi-time-prediction) as the name of the bucket

In [17]:
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope
import pathlib

In [18]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [19]:
def objective(params):
    with mlflow.start_run(nested=True):
         
        # Tag model
        mlflow.set_tag("model_family", "xgboost")
        
        # Log parameters
        mlflow.log_params(params)
        
        # Train model
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=100,
            evals=[(valid, 'validation')],
            early_stopping_rounds=10
        )
        
        # Log xgboost model with artifact_path
        mlflow.xgboost.log_model(booster, artifact_path="model")
         
        # Predict in the val dataset
        y_pred = booster.predict(valid)
        
        # Calculate metric
        rmse = root_mean_squared_error(y_val, y_pred)
        
        # Log performance metric
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [20]:
mlflow.xgboost.autolog()

with mlflow.start_run(run_name="Xgboost Hyper-parameter Optimization", nested=True):
    search_space = {
        'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
        'learning_rate': hp.loguniform('learning_rate', -3, 0),
        'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
        'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
        'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
        'objective': 'reg:squarederror',
        'seed': 42
    }
    
    best_params = fmin(
        fn=objective,
        space=search_space,
        algo=tpe.suggest,
        max_evals=10,
        trials=Trials()
    )
    best_params["max_depth"] = int(best_params["max_depth"])
    best_params["seed"] = 42
    best_params["objective"] = "reg:squarederror"
    
    mlflow.log_params(best_params)

    # Log tags
    mlflow.set_tags(
        tags={
            "project": "NYC Taxi Time Prediction Project",
            "optimizer_engine": "hyper-opt",
            "model_family": "xgboost",
            "feature_set_version": 1,
        }
    )

    # Log a fit model instance
    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=100,
        evals=[(valid, 'validation')],
        early_stopping_rounds=10
    )
        
    y_pred = booster.predict(valid)
    
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)
    
    pathlib.Path("models").mkdir(exist_ok=True)
    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
        
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

[0]	validation-rmse:7.28564                           
[1]	validation-rmse:6.28408                           
[2]	validation-rmse:5.76805                           
[3]	validation-rmse:5.50354                           
[4]	validation-rmse:5.38015                           
[5]	validation-rmse:5.30466                           
[6]	validation-rmse:5.25760                           
[7]	validation-rmse:5.23074                           
[8]	validation-rmse:5.21773                           
[9]	validation-rmse:5.20906                           
[10]	validation-rmse:5.20599                          
[11]	validation-rmse:5.20437                          
[12]	validation-rmse:5.20359                          
[13]	validation-rmse:5.20210                          
[14]	validation-rmse:5.20156                          
[15]	validation-rmse:5.19946                          
[16]	validation-rmse:5.19709                          
[17]	validation-rmse:5.19643                          
[18]	valid

2024/09/17 21:45:13 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/erick/Library/CloudStorage/OneDrive-ITESO/Proyecto_Ciencia_datos/nyc-taxi-time-prediction/.venv/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [21:45:13] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."

/Users/erick/Library/CloudStorage/OneDrive-ITESO/Proyecto_Ciencia_datos/nyc-taxi-time-prediction/.venv/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [21:45:18] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)

2024/09/17 21:45:20 WARNING mlflow.models.model: Model logged without a signature and input example. Pl

[0]	validation-rmse:8.33648                                                    
[1]	validation-rmse:7.70656                                                    
[2]	validation-rmse:7.19890                                                    
[3]	validation-rmse:6.79457                                                    
[4]	validation-rmse:6.47433                                                    
[5]	validation-rmse:6.22319                                                    
[6]	validation-rmse:6.02869                                                    
[7]	validation-rmse:5.87684                                                    
[8]	validation-rmse:5.75821                                                    
[9]	validation-rmse:5.66787                                                    
[10]	validation-rmse:5.59719                                                   
[11]	validation-rmse:5.54309                                                   
[12]	validation-rmse:5.50173            

2024/09/17 21:45:38 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/erick/Library/CloudStorage/OneDrive-ITESO/Proyecto_Ciencia_datos/nyc-taxi-time-prediction/.venv/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [21:45:38] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."

/Users/erick/Library/CloudStorage/OneDrive-ITESO/Proyecto_Ciencia_datos/nyc-taxi-time-prediction/.venv/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [21:45:42] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)

2024/09/17 21:45:43 WARNING mlflow.models.model: Model logged without a signature and input example. Pl

[0]	validation-rmse:8.63747                                                    
[1]	validation-rmse:8.21813                                                    
[2]	validation-rmse:7.84850                                                    
[3]	validation-rmse:7.52414                                                    
[4]	validation-rmse:7.23893                                                    
[5]	validation-rmse:6.98953                                                    
[6]	validation-rmse:6.77335                                                    
[7]	validation-rmse:6.58643                                                    
[8]	validation-rmse:6.42130                                                    
[9]	validation-rmse:6.28031                                                    
[10]	validation-rmse:6.16023                                                   
[11]	validation-rmse:6.05417                                                   
[12]	validation-rmse:5.96466            

...

2024/09/17 21:48:13 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/erick/Library/CloudStorage/OneDrive-ITESO/Proyecto_Ciencia_datos/nyc-taxi-time-prediction/.venv/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [21:48:13] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."

/Users/erick/Library/CloudStorage/OneDrive-ITESO/Proyecto_Ciencia_datos/nyc-taxi-time-prediction/.venv/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [21:48:18] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)

2024/09/17 21:48:19 WARNING mlflow.models.model: Model logged without a signature and input example. Pl

[0]	validation-rmse:5.59983                                                    
[1]	validation-rmse:5.56155                                                    
[2]	validation-rmse:5.54259                                                    
[3]	validation-rmse:5.53770                                                    
[4]	validation-rmse:5.52623                                                    
[5]	validation-rmse:5.51970                                                    
[6]	validation-rmse:5.51580                                                    
[7]	validation-rmse:5.50982                                                    
[8]	validation-rmse:5.50323                                                    
[9]	validation-rmse:5.48534                                                    
[10]	validation-rmse:5.48137                                                   
[11]	validation-rmse:5.47760                                                   
[12]	validation-rmse:5.47164            

2024/09/17 21:48:28 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/erick/Library/CloudStorage/OneDrive-ITESO/Proyecto_Ciencia_datos/nyc-taxi-time-prediction/.venv/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [21:48:28] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."

/Users/erick/Library/CloudStorage/OneDrive-ITESO/Proyecto_Ciencia_datos/nyc-taxi-time-prediction/.venv/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [21:48:32] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)

2024/09/17 21:48:33 WARNING mlflow.models.model: Model logged without a signature and input example. Pl

100%|██████████| 10/10 [03:35<00:00, 21.51s/trial, best loss: 5.160748647428987]
[0]	validation-rmse:7.28564
[1]	validation-rmse:6.28408
[2]	validation-rmse:5.76805
[3]	validation-rmse:5.50354
[4]	validation-rmse:5.38015
[5]	validation-rmse:5.30466
[6]	validation-rmse:5.25760
[7]	validation-rmse:5.23074
[8]	validation-rmse:5.21773
[9]	validation-rmse:5.20906
[10]	validation-rmse:5.20599
[11]	validation-rmse:5.20437
[12]	validation-rmse:5.20359
[13]	validation-rmse:5.20210
[14]	validation-rmse:5.20156
[15]	validation-rmse:5.19946
[16]	validation-rmse:5.19709
[17]	validation-rmse:5.19643
[18]	validation-rmse:5.19523
[19]	validation-rmse:5.19282
[20]	validation-rmse:5.18927
[21]	validation-rmse:5.18792
[22]	validation-rmse:5.18714
[23]	validation-rmse:5.18542
[24]	validation-rmse:5.18574
[25]	validation-rmse:5.18516
[26]	validation-rmse:5.18372
[27]	validation-rmse:5.17706
[28]	validation-rmse:5.17343
[29]	validation-rmse:5.17092
[30]	validation-rmse:5.17129
[31]	validation-rmse:5.16947
[

2024/09/17 21:48:47 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/erick/Library/CloudStorage/OneDrive-ITESO/Proyecto_Ciencia_datos/nyc-taxi-time-prediction/.venv/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [21:48:47] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."
2024/09/17 21:48:52 INFO mlflow.tracking._tracking_service.client: 🏃 View run Xgboost Hyper-parameter Optimization at: https://dagshub.com/daduke1/nyc-taxi-time-prediction.mlflow.mlflow/#/experiments/0/runs/d2ed9f3eb0f74993a97ef1c2df5cfe1c.
2024/09/17 21:48:52 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/daduke1/nyc-taxi-time-prediction.mlflow.mlflow/#/experiments/0.


In [21]:
run_id = input("Ingrese el run_id")
run_uri = f"runs:/{run_id}/model"

result = mlflow.register_model(
    model_uri=run_uri,
    name="nyc-taxi-model"
)

Successfully registered model 'nyc-taxi-model'.
2024/09/17 21:51:26 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: nyc-taxi-model, version 1
Created version '1' of model 'nyc-taxi-model'.


In [22]:
from datetime import datetime
from mlflow import MlflowClient

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)
client.update_registered_model(
    name="nyc-taxi-model",
    description="Model registry for the NYC Taxi Time Prediction Project",
)

new_alias = "champion"
date = datetime.today()
model_version = "1"

# create "champion" alias for version 1 of model "nyc-taxi-model"
client.set_registered_model_alias(
    name="nyc-taxi-model",
    alias=new_alias,
    version=model_version
)

client.update_model_version(
    name="nyc-taxi-model",
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_alias} on {date}",
)

<ModelVersion: aliases=['champion'], creation_timestamp=1726631486318, current_stage='None', description='The model version 1 was transitioned to champion on 2024-09-17 21:51:33.160526', last_updated_timestamp=1726631493562, name='nyc-taxi-model', run_id='f6dd9579fdfc4da9aac7bc1b56fc35a2', run_link='', source='mlflow-artifacts:/f141c1d9a6814959aec4b2f9fd9e18e0/f6dd9579fdfc4da9aac7bc1b56fc35a2/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>

In [23]:
import mlflow.pyfunc

model_name = "nyc-taxi-model"
alias = "champion"

model_uri = f"models:/{model_name}@{alias}"

champion_version = mlflow.pyfunc.load_model(
    model_uri=model_uri
)

champion_version.predict(X_val)

array([19.116783, 28.28602 ,  9.180469, ..., 46.627   , 13.745903,
       19.665049], dtype=float32)